In [1]:
# Este entorno de Python 3 es similar a Jupyter Notebook
# Viene con varias librerías instaladas. Para más információn podés consultar 
# la imagen de docker que utiliza (https://github.com/kaggle/docker-python)

import numpy as np 
import pandas as pd
import keras
import matplotlib.pyplot as plt
%matplotlib inline
import os


Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import load_img
dataset_folderpath="/kaggle/input/aa2019unlp/chest_xray"
print("Hay 3 directorios con los tres subconjuntos de datos: ",os.listdir(dataset_folderpath))
print("La carpeta de cada subconjunto tiene dos subcarpetas: NORMAL y PNEUMONIA")

train_folderpath = os.path.join(dataset_folderpath,"train")
val_folderpath = os.path.join(dataset_folderpath,"val")
test_folderpath = os.path.join(dataset_folderpath,"test")

Hay 3 directorios con los tres subconjuntos de datos:  ['test', 'val', 'train']
La carpeta de cada subconjunto tiene dos subcarpetas: NORMAL y PNEUMONIA


In [3]:
from keras.preprocessing.image import ImageDataGenerator
# mismo preprocesamiento que el usado originalmente para entrenar MobileNet
from keras.applications.mobilenet import preprocess_input

# Tamaño objetivo para escalar las imágenes. 
h,w,c = 224, 224, 3 # mismo tamaño que el usado originalmente para entrenar MobileNet

batch_size=32

# Preprocesamiento de cada subconjunto
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input # mismo preprocesamiento que el usado originalmente para entrenar MobileNet
    )

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input # mismo preprocesamiento que el usado originalmente para entrenar MobileNet
    )

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input # mismo preprocesamiento que el usado originalmente para entrenar MobileNet
    )

# Generadores de los subconjuntos. Reciben un directorio, y 
# cada carpeta del directorio se interpreta como una clase distinta.
# En este caso como cada directorio tiene 2 subdirectorios, NORMAL y PNEUMONIA,
# por ende, habrá dos clases.
# Además, al especificar el "class_mode" como binary, la salida se codifica como un solo valor
# (0 o 1), y no como un vector one-hot de dos elementos.

train_generator = train_datagen.flow_from_directory(
    train_folderpath, # directorio de donde cargar las imagenes (train)
    target_size=(h,w),
    batch_size=batch_size,
    class_mode='binary')

val_generator = val_datagen.flow_from_directory(
    val_folderpath, # directorio de donde cargar las imagenes (val)
    target_size=(h,w),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_folderpath,# directorio de donde cargar las imagenes (test)
    target_size=(h,w),
    batch_size=batch_size,
    class_mode=None, # IMPORTANTE ya que los ej de test no tienen clase
    shuffle=False # IMPORTANTE ya que nos importa el orden para el archivo de submission
    )


n_train=train_generator.samples
n_val=val_generator.samples
n_test=test_generator.samples
n_clases=train_generator.num_classes
print(f"Los conjuntos de train, val y test tienen {n_train}, {n_val} y {n_test} ejemplos respectivamente.")
print(f"Los conjuntos de datos tienen {n_clases} clases.")

Found 5088 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Found 512 images belonging to 1 classes.
Los conjuntos de train, val y test tienen 5088, 256 y 512 ejemplos respectivamente.
Los conjuntos de datos tienen 2 clases.


In [4]:
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.mobilenet import MobileNet

base_model=MobileNet(input_shape=(224,224,3),weights='imagenet',include_top=False) 
for layer in base_model.layers:
    layer.trainable=False # capas “congeladas” no se entrenan
output = GlobalAveragePooling2D()(base_model.output)    
# Utilizar salida del modelo como entrada a capa Dense de 128 
output=Dense(128,activation='relu')(output)
# Nueva capa de salida
output=Dense(1,activation='sigmoid')(output)

# Crear nuevo modelo en base a lo anterior
model=Model(inputs=base_model.input,outputs=output)

print(model.summary())

17227776/17225924 [==============================] - 1s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
____________________________________________________________

In [5]:
from keras import backend as K
# Definición de las métricas F1, recall y precision utilizando Keras.

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [6]:
val_steps=max(1,n_val // batch_size)
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy",f1_m,precision_m, recall_m])
model.fit_generator(train_generator,
                    steps_per_epoch=n_train // batch_size,
                    epochs=2,
                    validation_data=val_generator,
                    validation_steps=val_steps
                    )

Epoch 1/2
159/159 [==============================] - 321s 2s/step - loss: 0.1996 - accuracy: 0.9153 - f1_m: 0.9436 - precision_m: 0.9399 - recall_m: 0.9534 - val_loss: 0.4099 - val_accuracy: 0.7500 - val_f1_m: 0.7918 - val_precision_m: 0.6684 - val_recall_m: 0.9857
Epoch 2/2
159/159 [==============================] - 315s 2s/step - loss: 0.1367 - accuracy: 0.9473 - f1_m: 0.9652 - precision_m: 0.9628 - recall_m: 0.9706 - val_loss: 0.1944 - val_accuracy: 0.9062 - val_f1_m: 0.9112 - val_precision_m: 0.8711 - val_recall_m: 0.9625


In [7]:
#Evaluar el accuracy del modelo en el conjunto entero de entrenamiento
print("*********** Conjunto de entrenamiento ***********")
train_generator.reset()
asd = model.predict_generator(train_generator,steps=n_train // batch_size)
scores = model.evaluate_generator(train_generator,steps=n_train // batch_size)
for metric,score in zip(model.metrics_names,scores):
    print(f"{metric}: {score:.2f}")

print()
# Evaluar el accuracy del modelo en el conjunto entero de validación
print("*********** Conjunto de validación ***********")
val_generator.reset()
asd = model.predict_generator(val_generator,steps=n_val // batch_size)
scores = model.evaluate_generator(val_generator,steps=n_val // batch_size)
for metric,score in zip(model.metrics_names,scores):
    print(f"{metric}: {score:.2f}")


*********** Conjunto de entrenamiento ***********
loss: 0.25
accuracy: 0.91
f1_m: 0.94
precision_m: 0.94
recall_m: 0.95

*********** Conjunto de validación ***********
loss: 0.30
accuracy: 0.91
f1_m: 0.91
precision_m: 0.87
recall_m: 0.96


In [8]:


def save_predictions(y_prob,filenames,treshold,output="solutions.csv"):
    # Convertir probabilidades a etiquetas con el umbral
    y_pred = (y_prob[:,0]>0.5).astype(int)
    # quitar el nombre de la carpeta del nombre de archivo
    filenames=[ os.path.basename(f) for f in filenames]
    # igual cant de archivos que de predicciones
    assert(len(y_pred)==len(filenames))
    # Generar CSV con las predicciones
    import csv
    with open(output, mode='w') as f:
        writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['Id','Expected'])
        print("Id, Expected")
        for f,y in zip(filenames,y_pred):
            print(f"{f}, {str(y)}")
            writer.writerow([f,str(y)])

# predecir sobre el conjunto de test y generar el csv resultante
y_prob = model.predict_generator(test_generator,steps=n_test // batch_size)
# Establecer un umbral
treshold=0.5
save_predictions(y_prob,test_generator.filenames,treshold)


Id, Expected
test_0000.jpg, 1
test_0001.jpg, 0
test_0002.jpg, 0
test_0003.jpg, 1
test_0004.jpg, 0
test_0005.jpg, 1
test_0006.jpg, 0
test_0007.jpg, 0
test_0008.jpg, 1
test_0009.jpg, 1
test_0010.jpg, 1
test_0011.jpg, 0
test_0012.jpg, 0
test_0013.jpg, 1
test_0014.jpg, 0
test_0015.jpg, 1
test_0016.jpg, 0
test_0017.jpg, 0
test_0018.jpg, 0
test_0019.jpg, 1
test_0020.jpg, 1
test_0021.jpg, 0
test_0022.jpg, 1
test_0023.jpg, 0
test_0024.jpg, 0
test_0025.jpg, 0
test_0026.jpg, 0
test_0027.jpg, 1
test_0028.jpg, 0
test_0029.jpg, 1
test_0030.jpg, 1
test_0031.jpg, 1
test_0032.jpg, 1
test_0033.jpg, 1
test_0034.jpg, 1
test_0035.jpg, 1
test_0036.jpg, 1
test_0037.jpg, 1
test_0038.jpg, 1
test_0039.jpg, 1
test_0040.jpg, 0
test_0041.jpg, 1
test_0042.jpg, 1
test_0043.jpg, 1
test_0044.jpg, 0
test_0045.jpg, 1
test_0046.jpg, 0
test_0047.jpg, 1
test_0048.jpg, 0
test_0049.jpg, 1
test_0050.jpg, 1
test_0051.jpg, 1
test_0052.jpg, 1
test_0053.jpg, 0
test_0054.jpg, 1
test_0055.jpg, 1
test_0056.jpg, 0
test_0057.jpg, 0
t